# Introduction to Chain of Thought Reasoning

Chain of Thought (CoT) reasoning is a technique that guides language models to solve complex problems by breaking them down into intermediate steps. Instead of providing a direct answer, the model is prompted to "think aloud," generating a sequence of logical steps that lead to the final solution. This approach improves accuracy and interpretability, especially for tasks that require multi-step reasoning.

Let's see some basic examples.

In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(find_dotenv(), override=True)

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

Let's start with the minimal version of the Chain of Thought (CoT): Zero-shot CoT. Directly instruct so that it goes step by step on the process.

In [2]:
result = llm.invoke("What is the value of 5+7+9-12? \n Let's solve this step by step.")
print(result.content)

1. **5 + 7 = 12**
2. **12 + 9 = 21**
3. **21 - 12 = 9**

Therefore, the value of 5 + 7 + 9 - 12 is $\boxed{9}$.


So we can take a similar approximation and show examples on how to do this: Few-shot CoT.

In [3]:
result = llm.invoke(
    """
    Problem: What is the value of 3+4+19-12? 
    Solution: 
    Start with the first two numbers: 3+4 is 12. 
    Now add the next number to the result: 12+19 is 31. 
    Finally, subtract 12: 31-12 is 21. 
    So, the final answer is 21. 

    Problem: What is the value of 5+14+9+4+2? 
    Solution: 
    Start with the first two numbers: 5+14 is 19. 
    Now add the next number to the result: 19+9 is 28. 
    Again, add the next number to the result: 28+4=32. 
    Finally, add the last number: 32+2 is 34. 
    So, the final answer is 34. 

    Problem:  What is the value of 5+7+9-12? 
    """
)
print(result.content)

Solution:
Start with the first two numbers: 5+7 is 12.
Now add the next number to the result: 12+9 is 21.
Finally, subtract 12: 21-12 is 9.
So, the final answer is 9.


We can try to instruct better the LLM so that it creates a methodical process by interacting in different ways with the LLM. Like finding simpler examples to the initial query.

In [4]:
result_57 = llm.invoke("What is the value of 5+7? \n Let's solve this step by step.")

result_789 = llm.invoke("What is the value of 7*8-9? \n Let's solve this step by step.")

We can take this two examples and the process that the LLM has followed to exemplify for larger, more complex problems.

In [8]:
cot = f"""
    You are a math solving bot.

    Some examples on how to proceed:
    For 5+7 you can do {result_57.content}

    For 7*8-9 you can do {result_789.content}

    Problem:  What is the value of (5^2+5)/6?
"""

result = llm.invoke(cot)
print(result.content)

Following the order of operations (PEMDAS/BODMAS), we perform exponentiation, addition, and division.

1. **Exponentiation:** 5^2 = 25
2. **Addition:** 25 + 5 = 30
3. **Division:** 30 / 6 = 5

Therefore, the value of (5^2+5)/6 is $\boxed{5}$.
